collect_real_estate_api.ipynb
1. API 키 설정
2. API 호출 함수 작성
3. 기간별 데이터 수집
4. CSV 저장
5. Supabase 업로드

## 라이브러리 import

In [ ]:
import os
import requests
import time
import pandas as pd
import xml.etree.ElementTree as ET
from dotenv import load_dotenv

## 환경번수 로드

In [ ]:
load_dotenv('../.env')

print('환경변수 로드 완료')

## API 설정

In [ ]:
# .env 파일에서 API 키 가져오기
API_KEY = os.getenv('PUBLIC_DATA_API_KEY')

# 필수 파라미터
LAWD_CD = '1312313' # 지역코드(5자리: 12345)
DEAL_YMD = '20250101' # 계약월(6자리: 202401)

BASE_URL = f'https://apis.data.go.kr/1613000/RTMSDataSvcAptTrade/getRTMSDataSvcAptTrade?LAWD_CD{LAWD_CD}&DEAL_YMD={DEAL_YMD}&serviceKey={API_KEY};'

## API 호출 함수

In [ ]:
def get_apartment_trade(region_code, year_month):
    """
    아파트 실거래가 데이터를 가져오는 함수
    
    Parameters:
    - region_code: 시군구 코드 (예: 11680 = 강남구)
    - year_month: 연월 (예: 202401)
    
    Returns:
    - DataFrame: 실거래가 데이터
    """
    
    params = {
        'serviceKey': API_KEY,
        'LAWD_CD': region_code, # 지역코드
        'DEAL_YMD': year_month, # 계약월 (YYYYMM)
        # 'numOfRows': 1000,      # 한 번에 가져올 행 수
        # 'pageNo': 1             # 페이지 번호
    }

    try:
        print(f"API 호출: {region_code} / {year_month}")
        response = requests.get(BASE_URL, params=params)

        # 상태 코드 확인
        if response.status_code != 200:
            print(f"API 호출 실패: {response.status_code}")
            return None
        
        # XML파싱
        root = ET.fromstring(response.content)
        
        # resultCode 확인
        result_code = root.find('.//resultCode')
        if result_code is not None and result_code.text != '00':
            result_msg = root.find('.//resultMsg').text
            print(f"API 오류: {result_msg}")
            return None
        
        # 데이터 추출
        items = root.findall('.//item')

        if not items:
            print(f"ℹ데이터 없음: {region_code} / {year_month}")
            return None
        
        # 리스트로 변환
        data_list = []
        for item in items:
            data = {
                '거래금액': item.find('dealAmount').text.strip() if item.find('dealAmount') is not None else None,
                '건축년도': item.find('buildYear').text if item.find('buildYear') is not None else None,
                '년': item.find('dealYear').text if item.find('dealYear') is not None else None,
                '월': item.find('dealMonth').text if item.find('dealMonth') is not None else None,
                '일': item.find('dealDay').text if item.find('dealDay') is not None else None,
                '아파트명': item.find('aptNm').text if item.find('aptNm') is not None else None,
                '전용면적': item.find('excluUseAr').text if item.find('excluUseAr') is not None else None,
                '지번': item.find('jibun').text if item.find('jibun') is not None else None,
                '지역코드': item.find('rgionCd').text if item.find('rgionCd') is not None else None,
                '법정동': item.find('umdNm').text if item.find('umdNm') is not None else None,
                '층': item.find('floor').text if item.find('floor') is not None else None,
            }

            data_list.append(data)

            df = pd.DataFrame(data_list)
            print(f"{len(df)}건 수집 완료")

            return df

    except Exception as e:
        print(f'오류 발생: {e}')
        return None

## API 호출 및 CSV 생성

In [ ]:

print("\n" + "="*50)
print("테스트: 서울 강남구 2024년 1월 데이터 수집")
print("="*50 + "\n")

# 서울 강남구 (11680), 2024년 1월
test_df = get_apartment_trade('11680', '202401')

if test_df is not None:
    print("\n수집된 데이터 미리보기:")
    print(test_df.head())
    print(f"\n총 {len(test_df)}건 수집")
    
    # CSV로 저장
    test_df.to_csv('/home/claude/test_real_estate.csv', index=False, encoding='utf-8-sig')
    print("\n테스트 데이터 저장: test_real_estate.csv")

## 기간별 수집 예제

In [ ]:
def collect_period_data(region_code, start_ym, end_ym):
    """
    특정 기간의 데이터를 수집하는 함수
    
    Parameters:
    - region_code: 지역코드
    - start_ym: 시작 연월 (YYYYMM)
    - end_ym: 종료 연월 (YYYYMM)
    """
    all_data = []
    
    # 연월 리스트 생성
    start = pd.to_datetime(start_ym, format='%Y%m')
    end = pd.to_datetime(end_ym, format='%Y%m')
    months = pd.date_range(start, end, freq='MS')
    
    for month in months:
        ym = month.strftime('%Y%m')
        df = get_apartment_trade(region_code, ym)
        
        if df is not None:
            all_data.append(df)
        
        # API 호출 간격 (과부하 방지)
        time.sleep(2)
    
    if all_data:
        result_df = pd.concat(all_data, ignore_index=True)
        print(f"\n총 {len(result_df)}건 수집 완료")
        return result_df
    else:
        print("\n수집된 데이터 없음")
        return None


# 사용 예시 (주석 처리)
# df_2024 = collect_period_data('11680', '202401', '202412')

## 주요 지역코드 참고

In [ ]:
print("\n" + "="*50)
print("주요 지역코드 참고")
print("="*50)

region_codes = {
    '서울 강남구': '11680',
    '서울 강북구': '11305',
    '서울 강서구': '11500',
    '서울 관악구': '11620',
    '서울 광진구': '11215',
    '서울 구로구': '11530',
    '서울 금천구': '11545',
    '서울 노원구': '11350',
    '서울 도봉구': '11320',
    '서울 동대문구': '11230',
    '서울 동작구': '11590',
    '서울 마포구': '11440',
    '서울 서대문구': '11410',
    '서울 서초구': '11650',
    '서울 성동구': '11200',
    '서울 성북구': '11290',
    '서울 송파구': '11710',
    '서울 양천구': '11470',
    '서울 영등포구': '11560',
    '서울 용산구': '11170',
    '서울 은평구': '11380',
    '서울 종로구': '11110',
    '서울 중구': '11140',
    '서울 중랑구': '11260',
}

for name, code in region_codes.items():
    print(f"{name}: {code}")

print("\n" + "="*50)

In [ ]:
def collect_period_data(region_code, start_ym, end_ym):
    """
    특정 기간의 데이터를 수집하는 함수
    
    Parameters:
    - region_code: 지역코드
    - start_ym: 시작 연월 (YYYYMM)
    - end_ym: 종료 연월 (YYYYMM)
    """
    all_data = []
    
    # 연월 리스트 생성
    start = pd.to_datetime(start_ym, format='%Y%m')
    end = pd.to_datetime(end_ym, format='%Y%m')
    months = pd.date_range(start, end, freq='MS')
    
    for month in months:
        ym = month.strftime('%Y%m')
        df = get_apartment_trade(region_code, ym)
        
        if df is not None:
            all_data.append(df)
        
        # API 호출 간격 (과부하 방지)
        time.sleep(1)
    
    if all_data:
        result_df = pd.concat(all_data, ignore_index=True)
        print(f"\n총 {len(result_df)}건 수집 완료")
        return result_df
    else:
        print("\n수집된 데이터 없음")
        return None


# 사용 예시 (주석 처리)
# df_2024 = collect_period_data('11680', '202401', '202412')

In [ ]:
import time

# 대량 데이터 수집시에는 요청 사이 2초 정도의 대기 필요
for code in region_codes.values():
    df = get_apartment_trade(code, '202401')
    time.sleep(2)  # 2초 대기 (필수!)

# 2024년 전체 데이터 수집
df_2024 = collect_period_data('11680', '202401', '202412')

# CSV 저장
df_2024.to_csv('gangnam_2024.csv', index=False, encoding='utf-8-sig')

## 데이터 전처리

In [ ]:
# 거래금액 숫자로 변환
df['거래금액'] = df['거래금액'].str.replace(',', '').astype(int)

# 거래일자 생성
df['거래일자'] = pd.to_datetime(
    df['년'] + '-' + df['월'].str.zfill(2) + '-' + df['일'].str.zfill(2)
)

# 전용면적을 평수로 변환
df['전용면적_평'] = df['전용면적'].astype(float) / 3.3